<a href="https://colab.research.google.com/github/JANANI-B-S/oasis_infobytes/blob/main/Car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Load the dataset
data = pd.read_csv('/content/sample_data/car data.csv')

# Separate the features and target variable
X = data.drop(columns=['Selling_Price'])  # Features
y = data['Selling_Price']  # Target variable

# Identify categorical and numerical columns
categorical_features = [col for col in X.columns if X[col].dtype == 'object']
numerical_features = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessor for scaling and encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Define models
models = {
    'Linear Regression': Pipeline(steps=[('preprocessor', preprocessor),
                                         ('regressor', LinearRegression())]),

    'Random Forest': Pipeline(steps=[('preprocessor', preprocessor),
                                     ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])
}

# Train the model
model = models['Random Forest']  # Use Random Forest as an example
model.fit(X_train, y_train)

# Predict the model
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Results for Random Forest:")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}\n")

# Save the model
joblib.dump(model, 'car_price_model.pkl')

# Load the saved model for prediction
model = joblib.load('car_price_model.pkl')

# Function to get user input and predict the car price
def predict_car_price():
    car_name = input("Enter car name: ")
    year = int(input("Enter car year: "))
    present_price = float(input("Enter present price: "))
    driven_kms = int(input("Enter kilometers driven: "))
    fuel_type = input("Enter fuel type (Petrol/Diesel): ")
    selling_type = input("Enter selling type (Dealer/Individual): ")
    transmission = input("Enter transmission type (Manual/Automatic): ")
    owner = int(input("Enter number of owners: "))

    # Prepare the input for prediction
    user_input = pd.DataFrame({
        'Car_Name': [car_name],
        'Year': [year],
        'Present_Price': [present_price],
        'Driven_kms': [driven_kms],
        'Fuel_Type': [fuel_type],
        'Selling_type': [selling_type],
        'Transmission': [transmission],
        'Owner': [owner]
    })

    # Predict the selling price using the trained model
    predicted_price = model.predict(user_input)
    print(f"The predicted selling price of the car is: {predicted_price[0]:.2f} lakhs")

# Call the function to get user input and predict the price
predict_car_price()


Results for Random Forest:
Mean Absolute Error (MAE): 0.5950491803278689
Mean Squared Error (MSE): 0.7752991534426226
Root Mean Squared Error (RMSE): 0.880510734427822
R^2 Score: 0.9663434130460322

Enter car name: Honda Civic
Enter car year: 2016
Enter present price: 9.00
Enter kilometers driven: 30000
Enter fuel type (Petrol/Diesel): Petrol
Enter selling type (Dealer/Individual): Dealer
Enter transmission type (Manual/Automatic): Manual
Enter number of owners: 1
The predicted selling price of the car is: 7.35 lakhs
